# Getting Started!

This notebook shows how to get started with Quantus using time series data.
For this purpose, we use the 1D analogou of the MNIST dataset (Sam Greydanus):

https://github.com/greydanus/mnist1d

The model in this notebook is a CNN taken from the same repository:

https://github.com/greydanus/mnist1d/blob/master/models.py#L36

In [1]:
from IPython.display import clear_output

In [2]:
!python -m pip install quantus torch captum
 
#clear_output()

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import pathlib
import numpy as np
import pandas as pd

import quantus
from captum.attr import IntegratedGradients

import torch
import torch.nn as nn
from torch.utils.data import Dataset, Dataloader
torch.manual_seed(27)

clear_output()

np.random.seed(27)

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

ImportError: numpy.core.multiarray failed to import

## 1) Preliminaries

### 1.1 Load datasets

We load the dataset using the tensorflow-datasets library. Alternatively, it can be downloaded directly from the OpenML website: https://www.openml.org/d/40945

In [ ]:
import urllib.request
import pickle

mnist1d_data = pickle.load(urllib.request.urlopen("https://github.com/greydanus/mnist1d/blob/master/mnist1d_data.pkl?raw=true"))
mnist1d_data.keys()

In [ ]:
train_features = mnist1d_data['x']
train_labels = mnist1d_data['y']
test_features = mnist1d_data['x_test']
test_labels = mnist1d_data['y_test']

print(f'{train_features.shape = }')
print(f'{train_labels.shape = }')
print(f'{test_features.shape = }')
print(f'{test_labels.shape = }')


In [ ]:
# Create datasets


### 1.2 Train a model

The model is based on the model provided by the mnist1d repository:

https://github.com/greydanus/mnist1d/blob/master/models.py#L36

In [ ]:
class ConvBase(nn.Module):
    def __init__(self, output_size, channels=25, linear_in=125):
        super(ConvBase, self).__init__()
        self.conv1 = nn.Conv1d(1, channels, 5, stride=2, padding=1)
        self.conv2 = nn.Conv1d(channels, channels, 3, stride=2, padding=1)
        self.conv3 = nn.Conv1d(channels, channels, 3, stride=2, padding=1)
        self.linear = nn.Linear(linear_in, output_size) # flattened channels -> 10 (assumes input has dim 50)
        print("Initialized ConvBase model with {} parameters".format(self.count_params()))

    def count_params(self):
        return sum([p.view(-1).shape[0] for p in self.parameters()])

    def forward(self, x, verbose=False): # the print statements are for debugging
        x = x.view(-1,1,x.shape[-1])
        h1 = self.conv1(x).relu()
        h2 = self.conv2(h1).relu()
        h3 = self.conv3(h2).relu()
        h3 = h3.view(h3.shape[0], -1) # flatten the conv features
        return self.linear(h3) # a linear classifier goes on top

In [ ]:
net = ConvBase()

criterion = nn.CrossEntropyLoss()
num_epochs = 200

optimizer = torch.optim.Adam(net.parameters(), lr=0.1)
input_tensor = torch.from_numpy(train_features).type(torch.FloatTensor)
label_tensor = torch.from_numpy(train_labels)
for epoch in range(num_epochs):    
    output = net(input_tensor)
    loss = criterion(output, label_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 20 == 0:
        print ('Epoch {}/{} => Loss: {:.2f}'.format(epoch+1, num_epochs, loss.item()))

In [ ]:
out_probs = net(input_tensor).detach().numpy()
out_classes = np.argmax(out_probs, axis=1)
print("Train Accuracy:", sum(out_classes == train_labels) / len(train_labels))

In [ ]:
test_input_tensor = torch.from_numpy(test_features).type(torch.FloatTensor)
out_probs = net(test_input_tensor).detach().numpy()
out_classes = np.argmax(out_probs, axis=1)
print("Test Accuracy:", sum(out_classes == test_labels) / len(test_labels))

### 1.3 Generate explanations

In this example, we rely on the `captum` library. We use the Integrated Gradients method.

In [ ]:
ig = IntegratedGradients(net)

In [ ]:
test_input_tensor.requires_grad_()
attr, delta = ig.attribute(test_input_tensor,target=1, return_convergence_delta=True)
attr = attr.detach().numpy()

## 2) Quantative evaluation using Quantus

We can evaluate our explanations on a variety of quantuative criteria but as a motivating example we test the ModelParameterRandomisation scores by Adebayo et al., 2018. This metric measures the distance between the original attribution and a newly computed attribution throughout the process of cascadingly/independently randomizing the model parameters of one layer at a time.

In [ ]:
# Define metric for evaluation.
metric_init = quantus.ModelParameterRandomisation(
    similarity_func=quantus.similarity_func.correlation_spearman,
    return_sample_correlation=True,
    return_aggregate=True,
    aggregate_func=np.mean,
    layer_order="independent",
    disable_warnings=True,
    normalise=True,
    abs=True,)

In [ ]:
# Return ModelParameterRandomisation scores for Integrated Gradients.
scores_intgrad = metric_init(
    model=net, 
    x_batch=test_features,
    y_batch=test_labels,
    a_batch=None,
    explain_func=quantus.explain,
    explain_func_kwargs={
        "method": "IntegratedGradients",
        "reduce_axes": (),
    },
)

In [ ]:
print(f"ModelParameterRandomisation scores by Adebayo et al., 2018\n"       
      f"\n • Integrated Gradient = ",scores_intgrad)